# DataCollatorForLanguageModeling

## 1. Import 및 라이브러리 설치

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gc
import torch
import sys

gc.collect()
torch.cuda.empty_cache()

In [3]:
# 필요한 라이브러리 불러오기
import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    LineByLineTextDataset,
    EarlyStoppingCallback

)
from datasets import Dataset
from tqdm import tqdm

## 2. 모델 및 데이터 불러오기

### 1) 모델 불러오기

In [4]:
model_checkpoints = "gogamza/kobart-base-v2" #/MLM_pretrain_3ep_221121/checkpoint-33000"#"gogamza/kobart-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [5]:
def freeze_params(model):
    for par in model.parameters():
        par.requires_grad = False

In [6]:
freeze_params(model.get_encoder()) 

In [7]:
train_p = [p for p in model.parameters() if p.requires_grad] 
print(f'Length of train params in Summarization Model : {len(train_p)}')

Length of train params in Summarization Model : 159


### 2) 데이터 불러오기 및 확인

In [8]:
train_textfile_path = "data/train_text.csv"
val_textfile_path = "data/val_text.csv"

with open(train_textfile_path, encoding="utf-8") as f:
            train_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]     

with open(val_textfile_path, encoding="utf-8") as f:
            val_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]


In [9]:
del train_textlines[0]
del val_textlines[0]

In [10]:
train_df = pd.DataFrame(zip(train_textlines), columns=['Text'])
val_df = pd.DataFrame(zip(val_textlines), columns=['Text'])

In [11]:
train_df.reset_index(inplace=True, drop=True)
val_df.reset_index(inplace=True, drop=True)

In [12]:
train_df.head()

,Text
0,그럼 날짜는 가격 큰 변동 없으면 6 28-7 13로 확정할까 우리 비행포함 15일...
1,kf마스크만 5부제 하는거지 응 면마스크는 아무때나 사도될껀 면마스크말고 부직포 마...
2,아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애 그러...
3,칫솔사야하는데 쓱으로 살까 뭘 칫솔사는것까지 물어보시남 아 그 왕칫솔 또 사려나 싶...
4,잠도안오네 얼릉 고구마츄 먹고싶단 그게 그렇게 맛있었어 아주 여보 빼이보릿 되버렸네...


## 3. 데이터 토큰화

### 1) dataset으로 변환

In [13]:
train_data = Dataset.from_pandas(train_df)
val_data = Dataset.from_pandas(val_df)

### 2) EDA 바탕으로 길이 지정

In [14]:
max_input = 128
max_target = 128
batch_size = 16
ignore_index = -100

### 3) 토큰화 함수 구현 및 토큰화

In [15]:
def add_ignored_data(inputs, max_len, ignore_index):
    if len(inputs) < max_len:
        pad = [ignore_index] *(max_len - len(inputs)) # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
        inputs = np.concatenate([inputs, pad])
    else:
        inputs = inputs[:max_len]

    return inputs

In [16]:
def preprocess_data(data_to_process):
    label_id= []
    label_ids = []

    inputs = [dialogue for dialogue in data_to_process['Text']]
    model_inputs = tokenizer(inputs,  max_length=max_input, padding='max_length', truncation=True)

    for i in range(len(data_to_process['Text'])):
        label_id.append(tokenizer.encode(data_to_process['Text'][i]))  
    for i in range(len(data_to_process['Text'])):
        label_id[i].append(tokenizer.eos_token_id)
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))

    model_inputs['labels'] = label_ids

    return model_inputs

In [17]:
train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text'])

  0%|          | 0/280 [00:00<?, ?ba/s]

  0%|          | 0/36 [00:00<?, ?ba/s]

## 4. 학습을 진행하기 위한 준비

### 1) config 지정

In [18]:
model.config.decoder_start_token_id = tokenizer.bos_token_id                                             
model.config.eos_token_id = tokenizer.eos_token_id                        
model.config.max_length = 128
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 5
model.config.length_penalty = 2.0
model.config.num_beams = 5

### 2) rounge 함수 구현

In [19]:
rouge = datasets.load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
    rouge_output2 = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
    rouge_outputL = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid
    

    return {
        "rouge1_precision": round(rouge_output.precision, 4),
        "rouge1_recall": round(rouge_output.recall, 4),
        "rouge1_fmeasure": round(rouge_output.fmeasure, 4),
        
        "rouge2_precision": round(rouge_output2.precision, 4),
        "rouge2_recall": round(rouge_output2.recall, 4),
        "rouge2_fmeasure": round(rouge_output2.fmeasure, 4), 
        
        "rougeL_precision": round(rouge_outputL.precision, 4),
        "rougeL_recall": round(rouge_outputL.recall, 4),
        "rougeL_fmeasure": round(rouge_outputL.fmeasure, 4),
    }

/home/jx7789/anaconda3/envs/dohwan/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


### 3) arguments 지정

In [20]:
training_args = Seq2SeqTrainingArguments(
    output_dir="chckpoint/MLM_pretrain_basev2_freezing",
    num_train_epochs=50,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=64,
    learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    logging_dir="logs2",
    logging_steps=500,
    save_total_limit=10,
    #evaluation_strategy = "steps",# step별로 2버 loss가 오르는거 아니면 계속 반복하는듯
    #load_best_model_at_end = True,

)

### 4) data_collator

In [21]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

### 5) train

In [22]:
trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/home/jx7789/anaconda3/envs/dohwan/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 279992
  Num Epochs = 50
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 875000
  Number of trainable parameters = 57501696
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "tru

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jx7789. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

Deleting older checkpoint [chckpoint/MLM_pretrain_basev2_freezing/checkpoint-591500] due to args.save_total_limit
Saving model checkpoint to chckpoint/MLM_pretrain_basev2_freezing/checkpoint-597000
Configuration saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-597000/config.json
Model weights saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-597000/pytorch_model.bin
tokenizer config file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-597000/tokenizer_config.json
Special tokens file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-597000/special_tokens_map.json
Deleting older checkpoint [chckpoint/MLM_pretrain_basev2_freezing/checkpoint-592000] due to args.save_total_limit
Saving model checkpoint to chckpoint/MLM_pretrain_basev2_freezing/checkpoint-597500
Configuration saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-597500/config.json
Model weights saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-597500/pytorch_model.bin
toke

Model weights saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-603500/pytorch_model.bin
tokenizer config file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-603500/tokenizer_config.json
Special tokens file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-603500/special_tokens_map.json
Deleting older checkpoint [chckpoint/MLM_pretrain_basev2_freezing/checkpoint-598500] due to args.save_total_limit
Saving model checkpoint to chckpoint/MLM_pretrain_basev2_freezing/checkpoint-604000
Configuration saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-604000/config.json
Model weights saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-604000/pytorch_model.bin
tokenizer config file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-604000/tokenizer_config.json
Special tokens file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-604000/special_tokens_map.json
Deleting older checkpoint [chckpoint/MLM_pretrain_basev2_freezing/checkpoi

Deleting older checkpoint [chckpoint/MLM_pretrain_basev2_freezing/checkpoint-605000] due to args.save_total_limit
Saving model checkpoint to chckpoint/MLM_pretrain_basev2_freezing/checkpoint-610500
Configuration saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-610500/config.json
Model weights saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-610500/pytorch_model.bin
tokenizer config file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-610500/tokenizer_config.json
Special tokens file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-610500/special_tokens_map.json
Deleting older checkpoint [chckpoint/MLM_pretrain_basev2_freezing/checkpoint-605500] due to args.save_total_limit
Saving model checkpoint to chckpoint/MLM_pretrain_basev2_freezing/checkpoint-611000
Configuration saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-611000/config.json
Model weights saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-611000/pytorch_model.bin
toke

Model weights saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-617000/pytorch_model.bin
tokenizer config file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-617000/tokenizer_config.json
Special tokens file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-617000/special_tokens_map.json
Deleting older checkpoint [chckpoint/MLM_pretrain_basev2_freezing/checkpoint-612000] due to args.save_total_limit
Saving model checkpoint to chckpoint/MLM_pretrain_basev2_freezing/checkpoint-617500
Configuration saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-617500/config.json
Model weights saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-617500/pytorch_model.bin
tokenizer config file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-617500/tokenizer_config.json
Special tokens file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-617500/special_tokens_map.json
Deleting older checkpoint [chckpoint/MLM_pretrain_basev2_freezing/checkpoi

Deleting older checkpoint [chckpoint/MLM_pretrain_basev2_freezing/checkpoint-618500] due to args.save_total_limit
Saving model checkpoint to chckpoint/MLM_pretrain_basev2_freezing/checkpoint-624000
Configuration saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-624000/config.json
Model weights saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-624000/pytorch_model.bin
tokenizer config file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-624000/tokenizer_config.json
Special tokens file saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-624000/special_tokens_map.json
Deleting older checkpoint [chckpoint/MLM_pretrain_basev2_freezing/checkpoint-619000] due to args.save_total_limit
Saving model checkpoint to chckpoint/MLM_pretrain_basev2_freezing/checkpoint-624500
Configuration saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-624500/config.json
Model weights saved in chckpoint/MLM_pretrain_basev2_freezing/checkpoint-624500/pytorch_model.bin
toke

In [ ]:
trainer.evaluate()

## 5. 학습 결과 확인

In [ ]:
with open(val_textfile_path, encoding="utf-8") as f:
            val_textlines = [line for line in f.read().splitlines() if (len(line) > 0 and not line.isspace())]
        
val_df = pd.DataFrame(zip(val_textlines), columns=['Text'])
val_df.reset_index(inplace=True, drop=True)
val_data = Dataset.from_pandas(val_df)

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["Text"],
        padding="max_length",
        truncation=True,
        max_length=max_target,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, num_beams=5,no_repeat_ngram_size=5, max_length=128,
                            suppress_tokens= [234,23782,14338,240,199,198,161,116, 14338, 239], 
                             attention_mask=attention_mask, top_p=0.92)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)# 여기에 기본 kobart가져오기?
import random
from random import randrange
ck_num = len(val_data)
test_samples = val_data.select(range(0, ck_num, 1000))# 0, len(test_data), 200

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1] # 여기에 체크포인트 가져오기 
# 연구해봐야한다.

In [ ]:
for i in range(len(summaries_after_tuning)):
    print('idx_{} '.format(i))
    print("Summary before \n", summaries_before_tuning[i])
    print()
    print("Summary after \n", summaries_after_tuning[i])
    print()
    print("Target summary \n", test_samples["Summary"][i])
    print()
    print('Text', test_samples["Text"][i])
    print('-'*100)
    print()  